# EDA

# Load

In [ ]:
%run setup.ipynb

In [ ]:
from utils_kpi_station_density import (
    mask_active_stations,
    count_active_stations,
    transform_dataframe_to_folium_heatmap_data,
    normalize_weights,
    calc_density_in_grid
    )

In [ ]:
%store -r df stations

In [ ]:
df.head()

In [ ]:
stations.head()

In [ ]:
# city boundaries of washington
washington_boundary = gpd.read_file(config['raw_data_paths']['washington_outline'])
# washington_boundary = gpd.read_file('../data/raw/01_saopaulo_hdi.geojson')
washington_boundary = washington_boundary.drop(columns=['AREAMILES', 'OBJECTID', 'STATE_CITY', 'CAPITAL', 'WEB_URL', 'GLOBALID', 'CREATOR', 'CREATED', 'EDITOR','EDITED', 'SHAPEAREA', 'SHAPELEN'])
washington_boundary.head()

In [ ]:
washington_wards = gpd.read_file(config['raw_data_paths']['washington_wards'])
washington_wards = washington_wards.drop(columns=['LABEL', 'WARD', 'REP_NAME', 'WEB_URL', 'REP_PHONE', 'REP_EMAIL',
       'REP_OFFICE', 'STUSAB', 'SUMLEV', 'GEOID',
       'GEOCODE', 'STATE', 'OBJECTID', 'GLOBALID', 'CREATED_DATE',
       'LAST_EDITED_DATE', 'SHAPEAREA', 'SHAPELEN'])
washington_wards = washington_wards.set_index('WARD_ID')
washington_wards.columns = washington_wards.columns.str.lower()
washington_wards.head()

In [ ]:
# import metro stations
washington_metro_stations = gpd.read_file(config['raw_data_paths']['washington_metro_stations'])
washington_metro_stations.head()

In [ ]:
# import bicycle lanes
washington_bicycle_lanes = gpd.read_file(config['raw_data_paths']['washington_bicycle_lanes'])
washington_bicycle_lanes.head()

# Constants

In [ ]:
crs_standard = "EPSG:4326"

# Reproject to a Projected CRS for Area Calculation
# NOTE Geographic coordinates (lat/lon) are not suitable for direct area calculations.
#   Project to a local UTM zone or similar projected CRS.
#   Washington D.C. is in UTM Zone 18N (EPSG:32618)
crs_area_calculation = "EPSG:32618" # TODO why not 'epsg:32633'?

In [ ]:
# initialize the calculation
stations_gdf = gpd.GeoDataFrame(
    stations,
    geometry=[Point(lng, lat) for lat, lng in zip(stations.lat_median, stations.lng_median)],
    crs="EPSG:4326" # WGS 84 geographic coordinate system
)

# project the geometric data to the right crs
stations_gdf_proj = stations_gdf.to_crs(crs_area_calculation)
washington_boundary_proj = washington_boundary.to_crs(crs_area_calculation)
washington_wards_proj = washington_wards.to_crs(crs_area_calculation)

# Calculate areas
washington_boundary_proj["area"] = washington_boundary_proj['geometry'].area / 10**6 # km^2
washington_wards_proj["area"] = washington_wards_proj['geometry'].area / 10**6 # km^2

In [ ]:
fig = folium.Figure(width=800, height=800)
m = folium.Map(location=[LAT_DC, LNG_DC], zoom_start=12, tiles='CartoDB positron').add_to(fig)

folium.GeoJson(
    data=washington_boundary.to_json(), # Convert GeoDataFrame to GeoJSON string
    name='Boundary',       # Name for the layer in LayerControl
    style_function=lambda x: { # Customize the style of the ward polygons
        'fillColor': "#da2128",  # A brown-ish color
        'color': 'black',        # Border color
        'weight': 1,             # Border weight
        'fillOpacity': 0.1       # Transparency of the fill
    }
).add_to(m)

m

# KPI: Station Density

In [ ]:
# Create a map of all stations in DC
f = folium.Figure(width=800, height=800)
m = folium.Map(location=[LAT_DC, LNG_DC], zoom_start=12).add_to(f)

# Add markers for each bikeshare location
for index, row in stations.iterrows():
    folium.Marker(
        location=[row.lat_median, row.lng_median],
        #popup=f"Station: {row.get('name', 'N/A')}", # Use 'name' if available, otherwise 'N/A'
        icon=folium.Icon(color='blue', icon='bicycle', prefix='fa') # Add a bicycle icon
    ).add_to(m)

m

## Constants

In [ ]:
CELL_SIZE_METERS = 1000 # 1 km x 1 km grid cells

## Based on Area
1. Measure the service area
2. Calculate the stations per area
3. Build station density map

#### Overall Station Density over time

In [ ]:
# Overall station density
station_density_overall = stations.shape[0] / washington_boundary_proj.iloc[0].area # TODO how to make this more elegant? 
station_density_overall

In [ ]:
# station density over time
# it changes when a station is not serviced anymore
# exclude the last week or two weeks of the year: probably the stations have just not been targeted yet

station_density = pd.DataFrame(index = pd.date_range('2021.01.01.', '2023.12.31.', freq='D'))
res = station_density.index.to_series().apply(lambda x: count_active_stations(stations, x.date())) / washington_boundary_proj.iloc[0].area
station_density['density'] = res
station_density.head()

In [ ]:
# RuntimeError: Can not load face (invalid stream operation; error code 0x55)
import matplotlib
print(matplotlib.get_configdir())

In [ ]:
# plot station density over time
fig, ax = plt.subplots(1, 1)

station_density.plot(ax = ax)

ax.set_title('Station Density')
xlim = ax.get_xlim()
xlim = [pd.to_datetime(xlim[0], unit='D', origin='unix') + pd.Timedelta(days=21), pd.to_datetime(xlim[1], unit='D', origin='unix') - pd.Timedelta(days=14)]
# ax.set_xlim(xlim)
# ax.set_ylim([1.35, 1.7])

plt.tight_layout()
plt.show()

- The station density steadily increased over the years; from 1.4 to almost 1.7.
- The drops at the start and the end are edge effects: 

### Station Density Map

In [ ]:
# Create grid on Washington DC

# Bounding Box around the D.C. Boundary
# create a rectangular grid that covers this bounding box initially
xmin, ymin, xmax, ymax = washington_boundary_proj.total_bounds

# Generate coordinates for the grid lines
x_coords = np.arange(xmin, xmax + CELL_SIZE_METERS, CELL_SIZE_METERS)
y_coords = np.arange(ymin, ymax + CELL_SIZE_METERS, CELL_SIZE_METERS)

# Create a list of grid cell polygons
grid_cells = []
for i in range(len(x_coords) - 1):
    for j in range(len(y_coords) - 1):
        cell_polygon = Polygon([
            (x_coords[i], y_coords[j]),
            (x_coords[i + 1], y_coords[j]),
            (x_coords[i + 1], y_coords[j + 1]),
            (x_coords[i], y_coords[j + 1])
        ])
        grid_cells.append(cell_polygon)

# Create a GeoDataFrame from the grid cells
grid_gdf = gpd.GeoDataFrame(geometry=grid_cells, crs="EPSG:32618")

# cell area in km^2
grid_gdf['area_km2'] = grid_gdf.geometry.area / 10**6

# Clip the Grid to the D.C. Boundary
grid_in_dc = gpd.overlay(grid_gdf, washington_boundary_proj, how='intersection')

# Calculate area of each cell (all will be roughly cell_size_meters^2 for full cells)
# grid_in_dc['area_km2'] = grid_in_dc.geometry.area / 10**6 # Convert m^2 to km^2

In [ ]:
# Count Stations per Cell and Calculate Density
density_grid, stations_in_grid = calc_density_in_grid(stations_gdf_proj, grid_in_dc, pd.to_datetime('2021.01.01.').date())

# Density Map
# TODO visualize non-truncated grid
fig, ax = plt.subplots(1, 1, figsize=(10, 10))

# Plot the D.C. boundary (optional)
washington_boundary_proj.plot(ax=ax, color='lightgray', edgecolor='black', alpha=0.5)

# Plot the density map
density_grid.plot(column='density', ax=ax, legend=True,
                      cmap='YlOrRd', # Yellow-Orange-Red colormap
                      edgecolor='white', linewidth=0.5,
                      legend_kwds={'label': "Station Density (stations/km²)"})

# Plot the actual stations on top for context
stations_in_grid.plot(ax=ax, marker='o', color='blue', markersize=5, alpha=0.7)

ax.set_title('Public Transport Station Density in Washington D.C.')
ax.set_xlabel('Easting (m)')
ax.set_ylabel('Northing (m)')
plt.show()

In [ ]:
# stations density over time on a heatmap
intensify_factor = 1.2

f = folium.Figure(width=600, height=800)
m = folium.Map(location=[LAT_DC, LNG_DC], zoom_start=12).add_to(f)

# time frame
points_in_time = pd.date_range('2021.01.01.', '2023.12.31.', freq='ME').to_series()

# time resolved station density heatmap
time_series_df = [calc_density_in_grid(stations_gdf_proj, grid_in_dc, time.date())[0] for time in points_in_time]
time_series_df = normalize_weights(time_series_df, 'density')
data = [transform_dataframe_to_folium_heatmap_data(df, intensify_factor, crs_standard) for df in time_series_df]

hm = plugins.HeatMapWithTime(data,
                             name='Station Density per km2',
                             min_opacity=0.05, max_opacity=0.9,
                             min_speed=5, max_speed=10,
                             radius=50,
                             index=points_in_time.astype(str).to_list(),
                             auto_play=True,
                             ).add_to(m)

# layer: station locations
# fg = folium.FeatureGroup(name="Stations", show=False).add_to(m)
# folium.Marker(location=(LAT_DC, LNG_DC)).add_to(fg)

# ward layer
folium.GeoJson(
    data=washington_wards.to_json(), # Convert GeoDataFrame to GeoJSON string
    name='Wards',       # Name for the layer in LayerControl
    style_function=lambda x: { # Customize the style of the ward polygons
        'fillColor': '#8c510a',  # A brown-ish color
        'color': 'black',        # Border color
        'weight': 1,             # Border weight
        'fillOpacity': 0.2       # Transparency of the fill
    },
    tooltip=folium.features.GeoJsonTooltip( # Add tooltips on hover
        fields=['name'],
        aliases=['Ward Name:'],
        localize=True,
        sticky=False
    ),
    popup=folium.features.GeoJsonPopup( # Add popups on click
        fields=['name'],
        aliases=['Ward Name:'],
        localize=True,
        sticky=False
    )
).add_to(m)

folium.LayerControl().add_to(m)

# show map
m

In [ ]:
ward_ids_residential = set(['3', '4', '5', '7', '8'])
ward_ids_nonresidential = set(['1', '2', '6'])

### Station Density per Ward

- Ward 1, Ward 2 and Ward 6: well developed, non-residential wards
- Rest: residential wards

How has the station density evolved in these wards?

In [ ]:
# plotly express


# density_grid = calc_density_in_grid(stations_gdf_proj, grid_in_dc, pd.Timestamp('2021.01.01.').date())


# --- 3. Pre-calculate All Densities and Prepare Data for Plotly Express ---
# This is similar to the previous step, but we create a flat DataFrame for Plotly Express

time_frames = pd.to_datetime(pd.date_range('2021.01.01.', '2023.12.31.', freq='ME').to_series())

plotly_data_list = []
max_overall_density = 0
station_counts_time = 0
current_frame_grid_data = 0

for time_frame in time_frames:
    active_stations = stations_gdf_proj[mask_active_stations(stations_gdf_proj, time_frame.date())].copy()

    if not active_stations.empty:
        stations_in_full_grid = gpd.sjoin(active_stations, washington_wards_proj, how="inner", predicate='intersects')
        station_counts_time = stations_in_full_grid.groupby('WARD_ID').size().reset_index(name='station_count')
    else:
        station_counts_time = pd.DataFrame({'WARD_ID': [], 'station_count': []})

    current_frame_grid_data = washington_wards_proj.merge(
        station_counts_time, left_on='WARD_ID', right_on='WARD_ID', how='left'
    )
    current_frame_grid_data['station_count'] = current_frame_grid_data['station_count'].fillna(0).astype(int)
    current_frame_grid_data['density'] = current_frame_grid_data['station_count'] / current_frame_grid_data['area']

    # Append data for this year to the list for Plotly Express
    for idx, row in current_frame_grid_data.iterrows():
        plotly_data_list.append({
            'WARD_ID'   : row['WARD_ID'],
            'time'      : time_frame,
            'density'   : row['density']
        })
    
    if current_frame_grid_data['density'].max() > max_overall_density:
        max_overall_density = current_frame_grid_data['density'].max()

if max_overall_density == 0:
    max_overall_density = 1.0 # Avoid division by zero in color scaling

# Convert the list of dicts to a DataFrame
density_df_long = pd.DataFrame(plotly_data_list)
density_df_long.head()

In [ ]:
# Prepare GeoJSON for Plotly Express (only needs geometries and linking ID)
grid_geojson = json.loads(washington_wards.reset_index().to_json())

In [ ]:
# Create the Animated Choropleth Map
fig = px.choropleth_map(
    density_df_long,
    geojson=grid_geojson,
    locations='WARD_ID',                  # Column in DataFrame that links to GeoJSON
    featureidkey='properties.WARD_ID',    # Key in GeoJSON properties to match 'locations'
    color='density',                      # Column to color the polygons by
    animation_frame='time',               # Column to create the animation frames
    color_continuous_scale="Viridis",     # Colormap
    range_color=(0, 0.8*max_overall_density), # Consistent color scale across all frames
    zoom=10,                              # Initial zoom level
    center={"lat" : LAT_DC, "lon" : LNG_DC},
    opacity=0.7,
    labels={'density' : 'Station Density (stations/km²)'},
    title='Time-Resolved Station Density in Washington D.C.',
    width=1200,  # Set the width in pixels
    height=800  # Set the height in pixels
) 

# Customize animation speed (optional)
fig.layout.updatemenus[0].buttons[0].args[1]['frame']['duration'] = 500 # milliseconds per frame
fig.layout.updatemenus[0].buttons[0].args[1]['transition']['duration'] = 0 # no transition

# To ensure the animation slider always shows the whole range (optional, can be tricky with some datasets)
# fig.update_layout(sliders=[dict(steps=[dict(args=[[f.name]], method='animate') for f in fig.frames])])

fig.show()

# To save the map as an HTML file (interactive)
# fig.write_html("plotly_choropleth_time_resolved.html")

In [ ]:
fig, ax = plt.subplots(2, 1, figsize=(12,8), sharex=True)
ax = ax.flatten()

sns.lineplot(density_df_long[density_df_long.WARD_ID.isin(['1','2','6'])], x='time', y='density', hue='WARD_ID', ax=ax[0])
sns.lineplot(density_df_long[density_df_long.WARD_ID.apply(lambda x: x not in ['1','2','6'])], x='time', y='density', hue='WARD_ID', ax=ax[1])

plt.tight_layout()
plt.show()

- The station density per ward stayed almost constant until 2022-09. 
- The station density in the non-residential districts (Ward 1, 2 and 6) has slightly increased.
- The station density in the residential districts has slightly increased
- Significant increases in stations
  - 2022-08 in Wards 5, 6, 1
  - 2023-01 in Ward 7
- Slight increases in the other wards. 


## Based on Average distance 
1. station map: Sort stations on lat, lng grid
2. calc distance to neighbours
3. average

In [ ]:
# TODO

# Rides Distribution
- Which are the busiest stations? 
- Which are the most important connections? 

## Off-station rides

In [ ]:
# rides that start off-station
mask_ride_starts_off_station = df.start_station_id.isna()
print(sum(mask_ride_starts_off_station))
print(sum(mask_ride_starts_off_station)/df.shape[0])

In [ ]:
# rides that start off-station
mask_ride_ends_off_station = df.end_station_id.isna()
print(sum(mask_ride_ends_off_station))
print(sum(mask_ride_ends_off_station)/df.shape[0])

In [ ]:
# map rides that end off station
rides_end_off_station = df[mask_ride_ends_off_station]
rides_end_off_station.head()

In [ ]:
rides_end_off_station_gdf = gpd.GeoDataFrame(
    rides_end_off_station, 
    geometry=[Point(lng, lat) for lng, lat in zip(rides_end_off_station.end_lng, rides_end_off_station.end_lat)],
    crs = crs_standard
    )

rides_end_off_station_gdf = gpd.sjoin(rides_end_off_station_gdf, washington_wards, how='left', predicate='within')

In [ ]:
rides_end_off_station_gdf.head()

In [ ]:
# all rides that end off station
# data = rides_end_off_station_gdf.loc[(rides_end_off_station_gdf.ended_at.dt.year>2022), ['end_lat', 'end_lng']].groupby(['end_lat', 'end_lng']).size().rename('cnt').reset_index()
# data.cnt = scale_min_max(data.cnt)
# data = [[row.end_lat, row.end_lng, 100*np.log1p(row.cnt)] for i, row in data.iterrows() if row.cnt>0.0]

data = rides_end_off_station_gdf.loc[(rides_end_off_station_gdf.ended_at.dt.year>2022), ['end_lat', 'end_lng']]
data = [[row.end_lat, row.end_lng] for i, row in data.iterrows()]

fig = folium.Figure(width=800, height=800)
m = folium.Map(location=[LAT_DC, LNG_DC], zoom_start=12).add_to(fig)

hm = plugins.HeatMap(data,
                     name='end of off-station ride locations',
                     radius=25,
                     blur=25
                     ).add_to(m)

# Add markers for each bikeshare location
fg = folium.FeatureGroup(name="Station", show=False).add_to(m)
for index, row in stations.iterrows():
    pos_station = [row.lat_median, row.lng_median]
    # folium.Marker(
    #     location=pos_station,
    #     #popup=f"Station: {row.get('name', 'N/A')}", # Use 'name' if available, otherwise 'N/A'
    #     icon=folium.Icon(color='blue', icon='bicycle', prefix='fa') # Add a bicycle icon
    # ).add_to(fg)

    folium.Circle(
        pos_station,             
        # tooltip = "<b>Berghain</b>",
        radius = 10,                # Radius in meters 
        color = "black",
        fill = True,
        fill_color = "crimson"
    ).add_to(fg)

folium.LayerControl().add_to(m)

m

In [ ]:
rides_end_off_station_gdf.columns = rides_end_off_station_gdf.columns.str.lower()

In [ ]:
# member rides that end off station in residential wards
data = rides_end_off_station_gdf[(rides_end_off_station_gdf.ended_at.dt.year>2022) & 
                                 rides_end_off_station_gdf['ward_id'].isin(ward_ids_residential) & 
                                 (rides_end_off_station.member_casual=='member')]

data = [[row.end_lat, row.end_lng] for i, row in data.iterrows()]

# build data frame with bike locations that end off-station
fig = folium.Figure(width=800, height=800)
m = folium.Map(location=[LAT_DC, LNG_DC], zoom_start=12).add_to(fig)

hm = plugins.HeatMap(data,
                     name='end of off-station ride locations',
                     radius=25,
                     blur=25
                     ).add_to(m)

# Add markers for each bikeshare location
fg = folium.FeatureGroup(name="Station", show=False).add_to(m)
for index, row in stations.iterrows():
    pos_station = [row.lat_median, row.lng_median]
    # folium.Marker(
    #     location=pos_station,
    #     #popup=f"Station: {row.get('name', 'N/A')}", # Use 'name' if available, otherwise 'N/A'
    #     icon=folium.Icon(color='blue', icon='bicycle', prefix='fa') # Add a bicycle icon
    # ).add_to(fg)

    folium.Circle(
        pos_station,             
        # tooltip = "<b>Berghain</b>",
        radius = 10,                # Radius in meters 
        color = "black",
        fill = True,
        fill_color = "crimson"
    ).add_to(fg)

    
folium.GeoJson(
    data=washington_wards.to_json(), # Convert GeoDataFrame to GeoJSON string
    name='Wards',       # Name for the layer in LayerControl
    style_function=lambda x: { # Customize the style of the ward polygons
        'fillColor': '#8c510a',  # A brown-ish color
        'color': 'black',        # Border color
        'weight': 1,             # Border weight
        'fillOpacity': 0.2       # Transparency of the fill
    },
    tooltip=folium.features.GeoJsonTooltip( # Add tooltips on hover
        fields=['name'],
        aliases=['Ward Name:'],
        localize=True,
        sticky=False
    ),
    popup=folium.features.GeoJsonPopup( # Add popups on click
        fields=['name'],
        aliases=['Ward Name:'],
        localize=True,
        sticky=False
    )
).add_to(m)

folium.LayerControl().add_to(m)

m

In [ ]:
# redo the above map in static plt map

In [ ]:
print(sum(mask_ride_starts_off_station & mask_ride_ends_off_station))
print(sum(mask_ride_starts_off_station & mask_ride_ends_off_station)/df.shape[0])

- The fractions of rides that start off-station or that end off-station are quite small.
- Almost the same amount of bikes are picked up off station or are put off station.

Questions
- How is the development over time? -> possible KPI: station coverage
- How many bikes have to be brought back by capital bike share? 

In [ ]:
# daily rides that start off station
daily_rides_start_off = df.loc[mask_ride_starts_off_station, ['started_at', 'end_station_id']].groupby(df.started_at.dt.date).count().rename(columns={
    'started_at'        : 'count',
    'end_station_id'    : 'count_returned_to_station'
})

daily_rides_start_off['fraction_returned'] = daily_rides_start_off.count_returned_to_station / daily_rides_start_off['count']

daily_rides_start_off.head()

In [ ]:
# bikes that have been returned to a station by the users
fig, ax = plt.subplots(1, 1)

daily_rides_start_off.fraction_returned.plot(ax=ax)

ax.tick_params(axis='x', labelrotation=90)

plt.tight_layout()

# Station Graph

In [ ]:
import networkx as nx

### Build

In [ ]:
# collect dem data
graph_data = df[(df.rideable_type!='electric_bike') & 
                df.start_is_within_city & 
                (df.member_casual == 'member')
                ].groupby(['start_station_id', 'end_station_id']).agg(
    avg_duration=('ride_duration', 'mean'),
    ride_count=('start_station_id', 'size')
).reset_index().dropna(subset=['start_station_id', 'end_station_id'])
graph_data.sort_values(by='ride_count', ascending=False).head(100)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(12,6))

sns.histplot(graph_data.ride_count, ax=ax[0], bins=100, log_scale=True)
sns.histplot(graph_data.avg_duration, ax=ax[1], bins=100)

plt.tight_layout()

In [ ]:
# stations['cnt_net'] = stations.cnt_out.astype('int32[pyarrow]') - stations.cnt_in.astype('int32[pyarrow]')

In [ ]:
stations.head()

In [ ]:
# build graph
graph = nx.from_pandas_edgelist(
    graph_data,
    source='start_station_id',
    target='end_station_id',
    # edge_attr=['ride_count', 'avg_duration'],
    create_using=nx.DiGraph
    )

# edge attributes
nx.set_edge_attributes(graph, graph_data.set_index(['start_station_id', 'end_station_id']).ride_count.to_dict(), 'ride_count')
nx.set_edge_attributes(graph, graph_data.set_index(['start_station_id', 'end_station_id']).avg_duration.to_dict(), 'avg_duration')

# node attributes
nx.set_node_attributes(graph, stations.total_count_start_end.to_dict(), 'total_count_start_end')
# nx.set_node_attributes(graph, stations.cnt_in.to_dict(), 'cnt_in')
# nx.set_node_attributes(graph, stations.cnt_out.to_dict(), 'cnt_out')
# nx.set_node_attributes(graph, stations.cnt_net.to_dict(), 'cnt_net')
nx.set_node_attributes(graph, stations.lat_median.to_dict(), 'lat')
nx.set_node_attributes(graph, stations.lng_median.to_dict(), 'lon')

### Optimize

In [ ]:
# summarize: dedensify
c_graph, c_nodes = nx.dedensify(graph, threshold=500)
print(c_graph.number_of_edges())

### Analyse

In [ ]:
# Basic graph metrics
print(nx.number_of_nodes(graph))
print(nx.number_of_edges(graph))
print(graph.degree[0])
# print(nx.is_connected(graph))

In [ ]:
# Calculate various centrality metrics
# TODO What is the meaning of weight in all of these quantities? 
degree_cent         = nx.degree_centrality(graph)
betweenness_cent    = nx.betweenness_centrality(graph) # weights?
closeness_cent      = nx.closeness_centrality(graph, distance='ride_count') # might be interesting for analysis!
eigenvector_cent    = nx.eigenvector_centrality(graph)
clustering_coeff    = nx.clustering(graph, weight='ride_count')

In [ ]:
# set node attributes
nx.set_node_attributes(graph, eigenvector_cent, 'eigenvector_cent')

In [ ]:
# degree centrality: Degree centrality measures how many direct connections a node has 
# relative to the most possible connections in the network.
sns.histplot(pd.Series(degree_cent))
plt.tight_layout()

In [ ]:
# plot degrees
degrees = pd.Series([deg[1] for deg in graph.degree])
sns.histplot(degrees)
plt.tight_layout()

In [ ]:
# Clustering coefficient measures how close a node’s neighbors are to being a complete clique, 
# meaning how likely it is that two neighbors of a node are also directly connected.
sns.histplot(pd.Series(clustering_coeff))
plt.tight_layout()

In [ ]:
# edge betweenness on ride duration: find bottlenecks in rides
# does not really make sense
# edge_betweenness = nx.edge_betweenness_centrality(graph, k=nx.number_of_nodes(graph), weight='ride_duration')
# critical_connections = sorted(edge_betweenness.items(),
#                              key=lambda x: x[1], reverse=True)[:5]
# critical_connections

In [ ]:
# edge betweenness on inverse of ride count: Find bottlenecks in transport network
# edge_betweenness = nx.edge_betweenness_centrality(graph, k=nx.number_of_nodes(graph), weight='ride_count')
# critical_connections = sorted(edge_betweenness.items(),
#                              key=lambda x: x[1], reverse=True)[:5]
# critical_connections

In [ ]:
# community detection
# nx.community.greedy_modularity_communities(graph, weight='ride_count')
communities = nx.community.louvain_communities(graph, weight='ride_count')

### Visualize

#### Data Prep

In [ ]:
def get_most_important_edges(graph, attr_name, num_edges, self_looping=True):
    '''
    Create a subgraph from graph with the most important edges
    '''
    edges = set(graph.edges())
    if not self_looping:
        edges -= set(nx.selfloop_edges(graph))
    edges_with_weights = [(u, v, graph[u][v][attr_name]) for u, v in edges]
    # Sort edges by weight in descending order
    edges_with_weights.sort(key=lambda x: x[2], reverse=True)
    # Choose how many top edges to plot (e.g., top 5000)
    return edges_with_weights[:num_edges] 

def create_subgraph_with_largest_edges(graph, attr_name, num_edges, self_looping=True):
    '''
    Create a subgraph from graph with the most important edges
    '''
    edges = get_most_important_edges(graph, attr_name, num_edges, self_looping)
    return graph.edge_subgraph([(u, v) for u, v, w in edges])

def create_community_subgraph_with_largest_edges(graph, communities, attr_name, num_edges, self_looping=True):
    '''
    Create a subgraph from graph with the most important edges
    '''
    edges = list()
    for community in communities:
        subgraph = graph.subgraph(community)
        edges.extend(get_most_important_edges(subgraph,attr_name, num_edges, self_looping))
    # return [(u, v) for u, v, w in edges]
    return graph.edge_subgraph([(u, v) for u, v, w in edges])

In [ ]:
# get lat lon from graph
def get_lat_lon_from_graph(graph, name_lat, name_lon):
    pos = dict()
    for key, lat in nx.get_node_attributes(graph, name=name_lat).items():
        lon = nx.get_node_attributes(graph, name=name_lon)[key]
        pos.update({key : (lon, lat)})
    return pos

In [ ]:
# Choose how many top edges to plot (e.g., top 5000)
# num_edges_to_plot = 100
# graph_viz = create_subgraph_with_largest_edges(graph, 'ride_count', num_edges_to_plot, False)
num_edges_to_plot_per_comm = 50
graph_viz = create_community_subgraph_with_largest_edges(graph, communities, 'ride_count', num_edges_to_plot_per_comm, False)

In [ ]:
def scale_min_max(values: pd.Series):
    min_val = min(values)
    range_val = max(values)-min_val
    return (values-min_val)/range_val

def scale_to_range(values: pd.Series, min_val: float, max_val: float):
    values_norm = scale_min_max(values)
    return values_norm * (max_val-min_val) + min_val


In [ ]:
# node sizes
min_size = 10
max_size = 50

# set node sizes
node_size_viz = pd.Series({u: attr['total_count_start_end'] for u, attr in graph_viz.nodes(data=True)})
node_size_viz = scale_to_range(node_size_viz, min_size, max_size)
nx.set_node_attributes(graph_viz, node_size_viz.to_dict(), 'size')

In [ ]:
sns.scatterplot(node_size_viz)

In [ ]:
# node colors
color_map = {
   0 : "#d1780b",
   1 : "#0a2a8c",
   2 : "#12780b",
   3 : "#8c0a6b",
   4 : "#0dc4b8",
   5 : "#caad06"
}

def get_community_id(node_id, communities):
    for key, community in communities:
        if node_id in community:
            id = node_id
    return id

node_to_community_id = {}
for i, community_nodes in enumerate(communities):
    for node in community_nodes:
        node_to_community_id[node] = i

nx.set_node_attributes(graph_viz, node_to_community_id, 'community_id')

node_colors = [graph_viz.nodes[node]['community_id'] for node in graph_viz.nodes()]
edge_colors = np.add([np.sign(attr['eigenvector_cent']) for u, attr in graph_viz.nodes(data=True)], 2)

In [ ]:
node_colors_dict = {node : color_map[graph_viz.nodes[node]['community_id']] for node in graph_viz.nodes()}
# node_colors_dict

In [ ]:
# edge weights
min_weight = 1
max_weight = 10

weights = pd.Series({(u,v) : attr['ride_count'] for u,v, attr in graph_viz.edges(data=True)})
weights = scale_to_range(weights, min_weight, max_weight)
nx.set_edge_attributes(graph_viz, weights.to_dict(), 'weight')

#### Plots

In [ ]:
# community
fig, ax = plt.subplots(1, 1, figsize=(12,12))

# washington_boundary.plot(ax=ax, color='lightgray', edgecolor='black', alpha=0.5)
washington_wards.plot(ax=ax, color='lightgray', edgecolor='black', alpha=0.5)

pos = get_lat_lon_from_graph(graph_viz, 'lat', 'lon')#(attr['lat'], attr['lng']) for u, attr in graph_viz.nodes()]
# nx.draw(graph_viz, pos=pos, with_labels=False, ax=ax, node_size=10, alpha=0.4)
nx.draw_networkx_nodes(
    graph_viz,
    pos,
    node_size=node_size_viz,
    # edgecolors=edge_colors,
    node_color=node_colors,
    # cmap=plt.cm.Reds_r,
    ax=ax
)
nx.draw_networkx_edges(graph_viz, pos=pos, alpha=0.4, node_size=10, ax=ax)

plt.show()

In [ ]:
# Visualize directed graph
plt.figure(figsize=(12, 12))
# pos = nx.spring_layout(graph_viz)
pos = nx.layout.kamada_kawai_layout(graph_viz)
nx.draw_networkx(graph_viz,
                 pos,
                 arrowsize=15,
                 arrowstyle='-|>',
                 node_color='lightblue'
                 )

In [ ]:
nx.draw_shell(graph_viz, with_labels=True, font_weight='bold')

In [ ]:
# interactive visualization of the network 
from pyvis.network import Network

net = Network(height=800, width=1200, 
              directed=True, notebook=True, 
              # select_menu=True, filter_menu=True, 
              heading='Capital Bikeshare Station Network from 2021 - 2023'
              )
net.toggle_hide_nodes_on_drag(False)
net.barnes_hut()
net.from_nx(graph_viz,
            # edge_weight_transf=lambda x: x['ride_count'],
            show_edge_weights=True
            )

# net.set_options("""
# var options = {
#   "physics": {
#     "enabled": false
#   },
#   "layout": {
#     "improvedLayout": false
#   },
#   "nodes": {
#     "font": {
#       "size": 10
#     }
#   },
#   "edges": {
#     "width": 1,
#     "color": { "inherit": true }
#   }
# }
# """)

# net.prep_notebook()
net.show(name=config['reports_data_paths']['network_graph'], notebook=False)

In [ ]:
washington_metro_stations.drop(columns=['EDITED', 'CREATED']).info()

In [ ]:
# visualize the graph on a map
from folium.plugins import MarkerCluster # For Folium node clustering

m = folium.Map(location=[LAT_DC, LNG_DC], zoom_start=12)

# Add filtered edges as PolyLines
for u, v, attr in graph_viz.edges(data=True):
    weight          = attr['weight']
    start_coords    = [graph_viz.nodes[u]['lat'], G.nodes[u]['lon']]
    end_coords      = [graph_viz.nodes[v]['lat'], G.nodes[v]['lon']]
    folium.PolyLine(
        locations=[start_coords, end_coords],
        color='red',
        weight=weight, #/ 200, # Scale line thickness by weight (adjust divisor as needed)
        opacity=0.6,
        tooltip=f"Trips: {weight}" # Show trip count on hover
    ).add_to(m)

# plugins.AntPath(
#     locations=path_locations,
#     # You can add more options here
# ).add_to(m)
    
# Add nodes using MarkerCluster for performance
# marker_cluster = MarkerCluster(name='Stations').add_to(m)
fg_circle = folium.FeatureGroup(name='Stations').add_to(m)
for node_id, attrs in graph_viz.nodes(data=True):
    # folium.Marker(
    #     location=[attrs['lat'], attrs['lon']],
    #     tooltip=attrs.get('name', str(node_id)), # Show name on hover
    #     icon=folium.Icon(color='blue', icon='bicycle') # Optional icon
    # ).add_to(marker_cluster)
    
    folium.Circle(
        location=[attrs['lat'], attrs['lon']],
        radius = node_size_viz[node_id],                # Radius in meters 
        color = node_colors_dict[node_id],
        fill = True,
        stroke=False,
        fill_color = node_colors_dict[node_id],
        fill_opacity=1,
        opacity=1,
    ).add_to(fg_circle)


# bicycle lane layer
folium.GeoJson(
    data=washington_bicycle_lanes.to_json(), # Convert GeoDataFrame to GeoJSON string
    name='Bicycle Lanes',           # Name for the layer in LayerControl
    style_function=lambda x: {      # Customize the style of the ward polygons
    # 'fillColor': '#8c510a',       # A brown-ish color
        'color': 'blue',            # Border color
        'weight': 2,                # Border weight
        'fillOpacity': 0.2          # Transparency of the fill
    },
    show=False
).add_to(m)

folium.GeoJson(
    data=washington_metro_stations.drop(columns=['EDITED', 'CREATED']).to_json(), # Convert GeoDataFrame to GeoJSON string
    name='Metro stations',           # Name for the layer in LayerControl
    style_function=lambda x: {      # Customize the style of the ward polygons
    # 'fillColor': '#8c510a',       # A brown-ish color
        'color': 'green',            # Border color
        'weight': 2,                # Border weight
        'fillOpacity': 0.2          # Transparency of the fill
    },
    show=False
).add_to(m)

folium.LayerControl().add_to(m)

# Save the Folium map to an HTML file
folium_output_path = config['reports_data_paths']['network_graph_on_map']
m.save(folium_output_path)
print(f"\nFolium map saved to {folium_output_path}")

In [ ]:
import plotly.graph_objects as go # For Plotly

# # --- 4. Drawing with Plotly (for more customizable, publication-ready interactivity) ---

# # Prepare node data for Plotly
# node_lats = [attrs['lat'] for _, attrs in G.nodes(data=True)]
# node_lons = [attrs['lon'] for _, attrs in G.nodes(data=True)]
# node_names = [attrs['name'] for _, attrs in G.nodes(data=True)]

# # Prepare edge data for Plotly (list of lists/tuples for lines)
# edge_x = []
# edge_y = []
# edge_weights = []
# for u, v, weight in filtered_edges:
#     start_lat, start_lon = G.nodes[u]['lat'], G.nodes[u]['lon']
#     end_lat, end_lon = G.nodes[v]['lat'], G.nodes[v]['lon']
#     edge_x.extend([start_lon, end_lon, None]) # None creates a break between lines
#     edge_y.extend([start_lat, end_lat, None])
#     edge_weights.extend([weight, weight, None]) # Repeat weight for line segments

# fig = go.Figure()

# # Add Edges Layer
# fig.add_trace(go.Scattermapbox(
#     mode="lines",
#     lon=edge_x,
#     lat=edge_y,
#     # You can color/size edges based on weight here, using a custom hovertext
#     line=dict(
#         width=[w / 200 for w in edge_weights if w is not None], # Scale line thickness
#         color='blue', # Or scale color by weight using a colorscale
#         opacity=0.5
#     ),
#     hoverinfo="text",
#     text=[f"Trips: {w}" if w is not None else "" for w in edge_weights],
#     name="Edges"
# ))

# # Add Nodes Layer
# fig.add_trace(go.Scattermapbox(
#     mode="markers",
#     lon=node_lons,
#     lat=node_lats,
#     text=node_names, # Text for tooltip on hover
#     marker=dict(
#         size=8,
#         color='red', # You can also color nodes based on centrality, etc.
#         opacity=0.8
#     ),
#     hoverinfo='text',
#     name="Nodes"
# ))

# fig.update_layout(
#     mapbox_style="carto-positron", # Choose a map style (e.g., "open-street-map", "carto-darkmatter")
#     mapbox_zoom=11.5, # Adjust zoom level
#     mapbox_center={"lat": avg_lat, "lon": avg_lon},
#     title='Interactive Bikeshare Network on Map (Plotly)',
#     showlegend=False, # Hide legend if not needed for these simple traces
#     margin={"r":0,"t":40,"l":0,"b":0} # Adjust margins
# )

# # Save the Plotly map to an HTML file
# plotly_output_path = 'network_map_plotly.html'
# fig.write_html(plotly_output_path)
# print(f"Plotly map saved to {plotly_output_path}")

# Time Series Analysis for Daily Rides in a Community

In [ ]:
bdays = pd.Series(pd.bdate_range(start='2021.01.01', end='2024.01.01', inclusive='left').to_pydatetime()).dt.date
mask = ((df.ended_at.dt.year>2022) & 
        (df.member_casual=='casual') & 
        ~df.is_holiday & 
        df.end_is_within_city & 
        df.ended_at.dt.date.isin(bdays)
        )

# grouper_h = pd.Grouper(key='ended_at', level=None, freq='h', axis=0, sort=False)
hourly_rides_end = df[mask].groupby(by=[df.ended_at.dt.hour, 'end_station_id']).size().rename('ride_ends_cnt').reset_index()
hourly_rides_end = pd.merge(hourly_rides_end, stations[['lat_median', 'lng_median']], left_on=['end_station_id'], right_index=True, how='left')
hourly_rides_end = hourly_rides_end.rename(columns={'ended_at':'hour', 'end_station_id':'station_id', 'lat_median':'lat', 'lng_median':'lng'})
# hourly_rides_end

hourly_rides_start = df[mask].groupby(by=[df.started_at.dt.hour, 'start_station_id']).size().rename('ride_starts_cnt').reset_index()
hourly_rides_start = pd.merge(hourly_rides_start, stations[['lat_median', 'lng_median']], left_on=['start_station_id'], right_index=True, how='left')
hourly_rides_start = hourly_rides_start.rename(columns={'started_at':'hour', 'start_station_id':'station_id', 'lat_median':'lat', 'lng_median':'lng'})
# hourly_rides_start

In [ ]:
hourly_rides_end.head()

In [ ]:
def hourly_rides_2_map(time_series, col, intensify_factor, log_scale):
    if log_scale:
        return [[row.lat, row.lng, intensify_factor*np.log1p(row[col])] for _, row in time_series.iterrows() if row[col]>0.]
    else:
        return [[row.lat, row.lng, intensify_factor*row[col]] for _, row in time_series.iterrows() if row[col]>0.]

In [ ]:
# Time resolved Heatmap: end of ride density
# How are people moving around?

# stations density over time on a heatmap
intensify_factor = 1

f = folium.Figure(width=1000, height=1200)
m = folium.Map(location=[LAT_DC, LNG_DC], zoom_start=12).add_to(f)

# time resolved station density heatmap
points_in_time = pd.Series(range(24))
time_series_df = [hourly_rides_end[hourly_rides_end.hour == hour] for hour in points_in_time]
time_series_df = normalize_weights(time_series_df, 'ride_ends_cnt')

data = [hourly_rides_2_map(df, 'ride_ends_cnt', intensify_factor, True) for df in time_series_df]

hm = plugins.HeatMapWithTime(data,
                             name='End of Rides per hour',
                             min_opacity=0.05, max_opacity=0.9,
                             min_speed=5, max_speed=10,
                             radius=50,
                             index=points_in_time.astype(str).to_list(),
                             auto_play=True,
                             ).add_to(m)

m

In [ ]:
# stations density over time on a heatmap
intensify_factor = 1

f = folium.Figure(width=1000, height=1200)
m = folium.Map(location=[LAT_DC, LNG_DC], zoom_start=12).add_to(f)

# time resolved station density heatmap
points_in_time = pd.Series(range(24))
time_series_df = [hourly_rides_start[hourly_rides_start.hour == hour] for hour in points_in_time]
time_series_df = normalize_weights(time_series_df, 'ride_starts_cnt')

data = [hourly_rides_2_map(df, 'ride_starts_cnt', intensify_factor, True) for df in time_series_df]

hm = plugins.HeatMapWithTime(data,
                             name='Start of Rides per hour',
                             min_opacity=0.05, max_opacity=0.9,
                             min_speed=5, max_speed=10,
                             radius=50,
                             index=points_in_time.astype(str).to_list(),
                             auto_play=True,
                             ).add_to(m)

m

In [ ]:
# Time series analysis on Community
# Analyse the daily rides that connect the stations